In [ ]:
pip install kaggle

In [ ]:
mkdir ~/.kaggle

In [ ]:
mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d faber24/lcc-fasd

Dataset URL: https://www.kaggle.com/datasets/faber24/lcc-fasd
License(s): unknown
100% 4.83G/4.84G [00:33<00:00, 205MB/s]
100% 4.84G/4.84G [00:33<00:00, 156MB/s]


In [ ]:
!unzip lcc-fasd.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_167.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_30.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_45.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_60.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_75.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s0_90.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s2_107.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s2_122.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s2_137.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE5F_id47_s2_152.png  
  inflating: LCC_FASD/LCC_FASD_training/spoof/YOUTUBE_BQEW2730_IPHONE

In [ ]:
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = datasets.ImageFolder(root='/content/LCC_FASD/LCC_FASD_development', transform=transform)

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader for each split
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class FaceAntiSpoofingCNN(nn.Module):
    def __init__(self):
        super(FaceAntiSpoofingCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 2)  # Output: live/spoof classification (2 classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
my_nn = FaceAntiSpoofingCNN()

In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = FaceAntiSpoofingCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # You can change this
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set model to training mode
    for inputs, labels in train_loader:  # Assuming train_loader is defined
        # Move data to GPU if available
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        model = model.to('cuda')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation phase (optional)
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()

            _, predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip uninstall torch torchvision torchaudio --yes


Found existing installation: torch 2.4.0+cpu
Uninstalling torch-2.4.0+cpu:
  Successfully uninstalled torch-2.4.0+cpu
Found existing installation: torchvision 0.19.0+cpu
Uninstalling torchvision-0.19.0+cpu:
  Successfully uninstalled torchvision-0.19.0+cpu
Found existing installation: torchaudio 2.4.0+cpu
Uninstalling torchaudio-2.4.0+cpu:
  Successfully uninstalled torchaudio-2.4.0+cpu


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=1a170ddfb48d683c8e5dd58baa5f50020db524b05ce8f20f80404acf415535c4
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit


In [ ]:
# Remove the `.to('cuda')` parts
model = FaceAntiSpoofingCNN()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()  # Set model to training mode
    for inputs, labels in train_loader:
        # No need to move data to 'cuda'
        inputs, labels = inputs, labels

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation phase (optional)
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            # No need to move data to 'cuda'
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()

            _, predicted = torch.max(val_outputs.data, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')


Epoch 1, Loss: 0.6498178931382986
Validation Accuracy: 86.24787775891342%
Epoch 2, Loss: 0.6500069902493404
Validation Accuracy: 86.75721561969439%
Epoch 3, Loss: 0.6505631290949307
Validation Accuracy: 86.5874363327674%
Epoch 4, Loss: 0.6500554130627558
Validation Accuracy: 86.41765704584041%
Epoch 5, Loss: 0.6499743314889761
Validation Accuracy: 85.90831918505943%
Epoch 6, Loss: 0.6501110498721783
Validation Accuracy: 86.24787775891342%
Epoch 7, Loss: 0.6501599256808941
Validation Accuracy: 86.41765704584041%
Epoch 8, Loss: 0.650323933821458
Validation Accuracy: 85.90831918505943%
Epoch 9, Loss: 0.6502044723584102
Validation Accuracy: 86.41765704584041%
Epoch 10, Loss: 0.6500476525380061
Validation Accuracy: 85.56876061120543%


In [ ]:


# Save the model weights
model_state = model.state_dict()
torch.save(model_state, 'face_antispoof_cnn.pth')

In [ ]:
!pip install nada-ai

In [ ]:
!pip install git+https://github.com/NillionNetwork/nada-ai.git



  Cloning https://github.com/NillionNetwork/nada-ai.git to /tmp/pip-req-build-oxqw2gi6
  Running command git clone --filter=blob:none --quiet https://github.com/NillionNetwork/nada-ai.git /tmp/pip-req-build-oxqw2gi6
  Resolved https://github.com/NillionNetwork/nada-ai.git to commit 38e68a5cd1b75d6d44ccb0720a0096828619aac8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import nada

nada.export_model(model_state, "face_antispoof_cnn.pth")

ModuleNotFoundError: No module named 'nada'

In [ ]:
import os
import time
import sys
import uuid

In [ ]:
!nada init nada-face

Creating new nada project: nada-face
Project created!


In [ ]:
# Configure telemetry settings
enable_telemetry = True  #@param {type:"boolean"}
my_github_username = "your-github-username"  #@param {type:"string"}

In [ ]:
# Install the nilup tool and then use that to install the Nillion SDK
!curl https://nilup.nilogy.xyz/install.sh | bash

# Update Path if ran in colab
if "google.colab" in sys.modules:
    os.environ["PATH"] += ":/root/.nilup/bin"
    os.environ["PATH"] += ":/root/.nilup/sdks/latest/"

# Set telemetry if opted in
if enable_telemetry:
    identifier = f"nada-ai-conv-net-{str(uuid.uuid4())}-{my_github_username}"
    !echo 'yes' | nilup instrumentation enable --wallet {identifier}

# Install the lastest SDK and initialise it
!nilup init
!nilup install latest
!nilup use latest

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7810  100  7810    0     0  15158      0 --:--:-- --:--:-- --:--:-- 15135

nilup has been installed into /root/.nilup/bin and added to your $PATH in /root/.bashrc.

Run 'source /root/.bashrc' or start a new terminal session to use nilup.

By providing your Ethereum wallet address, you consent to the collection of telemetry data by the Nillion Network.
That includes but is not limited to
- The version of the SDK you are using
- The OS you are using
- The Processor Architecture you are using
- The SDK binary that you are running and the subcommand
- The wallet address you provided
- The errors produced by the SDK
We collect this data to understand how the software is used, and to better assist you in case of issues.
While we will not collect any personal information, we still recommend using a new wallet address that cannot be 

In [ ]:
# Spin up local Nillion devnet
!nohup nillion-devnet &

time.sleep(20)  # Wait for devnet

nohup: appending output to 'nohup.out'


In [ ]:
import nada_numpy as na
from nada_ai import nn, Party

class AntiSpoofingNN(nn.Module):
    """CNN for face anti-spoofing"""

    def __init__(self) -> None:
        """Model with two layers and activations"""
        super(AntiSpoofingNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)  # Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 2)  # Two output classes: live or spoof
        self.relu = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        """Forward pass logic"""
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 32 * 6 * 6)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def anti_spoofing_inference():
    # Step 1: Create "Provider" and "User" parties
    user = Party(name="User")
    provider = Party(name="Provider")

    # Step 2: Instantiate model object
    model = AntiSpoofingNN()

    # Step 3: Load model weights from Nillion network
    model.load_state_from_network("anti_spoofing_model", provider, na.SecretRational)

    # Step 4: Load input data (face image from User)
    input_image = na.array((3, 224, 224), user, "input_face_image", na.SecretRational)

    # Step 5: Compute inference
    result = model(input_image)

    # Step 6: Output result (live or spoof)
    return result.output(user, "anti_spoofing_output")



ImportError: cannot import name 'Party' from 'nada_ai' (/usr/local/lib/python3.10/dist-packages/nada_ai/__init__.py)

In [ ]:
import nada_numpy as na
from nada_ai import nn

In [ ]:
class AntiSpoofingNN(nn.Module):
    """CNN for face anti-spoofing"""

    def __init__(self) -> None:
        """Model with two layers and activations"""
        super(AntiSpoofingNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)  # Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 2)  # Two output classes: live or spoof
        self.relu = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        """Forward pass logic"""
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 32 * 6 * 6)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def anti_spoofing_inference():
    # Step 1: Create "Provider" and "User" parties
    user = Party(name="User")
    provider = Party(name="Provider")

    # Step 2: Instantiate model object
    model = AntiSpoofingNN()

    # Step 3: Load model weights from Nillion network
    model.load_state_from_network("anti_spoofing_model", provider, na.SecretRational)

    # Step 4: Load input data (face image from User)
    input_image = na.array((3, 224, 224), user, "input_face_image", na.SecretRational)

    # Step 5: Compute inference
    result = model(input_image)

    # Step 6: Output result (live or spoof)
    return result.output(user, "anti_spoofing_output")

In [ ]:
!nada build

Building program: inference
Error: 
   0: Traceback (most recent call last):
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 43, in compile
          main = getattr(script, "nada_main")
      AttributeError: module 'inference' has no attribute 'nada_main'

      During handling of the above exception, another exception occurred:

      Traceback (most recent call last):
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 97, in <module>
          output = compile(sys.argv[1])
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/timer.py", line 116, in wrapper
          value = func(*args, **kwargs)
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 45, in compile
          raise Exception(
      Exception: 'nada_dsl' entrypoint function is missing in program inference
   0: 

Location:
   tools/nada/src/error.rs:10

Backtrace omitted. Run with RUST_BACKTRACE=1 environment variable to d

In [ ]:
from IPython.core.magic import register_cell_magic
from pathlib import Path

In [ ]:
@register_cell_magic
def to_file(line, cell):
    "Writes the content of the cell to a file specified in the line argument."
    filepath = Path(line.strip())
    filepath.parent.mkdir(parents=True, exist_ok=True)

    with open(filepath, "w") as f:
        f.write(cell)

In [ ]:
%%to_file nada-project.toml
name = "inference"
version = "0.1.0"
authors = [""]

[[programs]]
path = "src/inference.py"
prime_size = 128

In [ ]:
%%to_file src/inference.py
import nada_numpy as na
from nada_ai import nn

class AntiSpoofingNN(nn.Module):
    """CNN for face anti-spoofing"""

    def __init__(self) -> None:
        """Model with two layers and activations"""
        super(AntiSpoofingNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)  # Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 2)  # Two output classes: live or spoof
        self.relu = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        """Forward pass logic"""
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 32 * 6 * 6)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def anti_spoofing_inference():
    # Step 1: Create "Provider" and "User" parties
    user = Party(name="User")
    provider = Party(name="Provider")

    # Step 2: Instantiate model object
    model = AntiSpoofingNN()

    # Step 3: Load model weights from Nillion network
    model.load_state_from_network("anti_spoofing_model", provider, na.SecretRational)

    # Step 4: Load input data (face image from User)
    input_image = na.array((3, 224, 224), user, "input_face_image", na.SecretRational)

    # Step 5: Compute inference
    result = model(input_image)

    # Step 6: Output result (live or spoof)
    return result.output(user, "anti_spoofing_output")



In [ ]:
!pip install --upgrade nada-dsl

In [ ]:
from nada_dsl import *

In [ ]:
from nada_dsl import *

def nada_main():
    # Define parties
    party1 = Party(name="User")
    party2 = Party(name="Provider")

    # Load face anti-spoofing model from party2 (provider)
    model = SecretModel(Input(name="anti_spoofing_model", party=party2))

    # Load input face image from party1 (user)
    face_image = SecretImage(Input(name="input_face_image", party=party1))

    # Run inference
    result = model(face_image)

    # Return the spoof detection result (live or spoof)
    return [Output(result, "anti_spoofing_output", party1)]


In [ ]:
!nada build

Building program: inference
Error: 
   0: Traceback (most recent call last):
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 43, in compile
          main = getattr(script, "nada_main")
      AttributeError: module 'inference' has no attribute 'nada_main'

      During handling of the above exception, another exception occurred:

      Traceback (most recent call last):
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 97, in <module>
          output = compile(sys.argv[1])
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/timer.py", line 116, in wrapper
          value = func(*args, **kwargs)
        File "/usr/local/lib/python3.10/dist-packages/nada_dsl/compile.py", line 45, in compile
          raise Exception(
      Exception: 'nada_dsl' entrypoint function is missing in program inference
   0: 

Location:
   tools/nada/src/error.rs:10

Backtrace omitted. Run with RUST_BACKTRACE=1 environment variable to d

In [ ]:
%%to_file src/inference.py
import nada_numpy as na
from nada_ai import nn
from nada_dsl import *

# Define Anti-Spoofing CNN Model
class AntiSpoofingNN(nn.Module):
    """CNN for face anti-spoofing"""

    def __init__(self) -> None:
        """Model with two layers and activations"""
        super(AntiSpoofingNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)  # Convolutional layer
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 2)  # Two output classes: live or spoof
        self.relu = nn.ReLU()

    def forward(self, x: na.NadaArray) -> na.NadaArray:
        """Forward pass logic"""
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 32 * 6 * 6)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def nada_main() -> List[Output]:
    """Main Nada DSL logic with secure computation"""
    # Define parties
    user = Party(name="User")
    provider = Party(name="Provider")

    # Step 2: Instantiate model object
    model = AntiSpoofingNN()

    # Load model weights from Nillion network
    model.load_state_from_network("anti_spoofing_model", provider, na.SecretRational)

    # Load input data (face image from User)
    input_image = na.array((3, 224, 224), user, "input_face_image", na.SecretRational)

    # Compute inference
    result = model(input_image)

    # Return the result to the user securely
    return [Output(result, "anti_spoofing_output", user)]


In [ ]:
!nada build

Building program: inference
^C
